In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, Column, vstack
import collections
%matplotlib inline


## Read HLC and NSC Tables

In [ ]:
#--------------HLC-----------------------#

hlc1=Table.read('HLC.RA_00_to_01.fits.gz')
hlc2=Table.read('HLC.RA_01_to_02.fits.gz')
hlc3=Table.read('HLC.RA_02_to_03.fits.gz')
hlc4=Table.read('HLC.RA_03_to_04.fits.gz')
hlc5=Table.read('HLC.RA_20_to_21.fits.gz')
hlc6=Table.read('HLC.RA_21_to_22.fits.gz')
hlc7=Table.read('HLC.RA_22_to_23.fits.gz')
hlc8=Table.read('HLC.RA_23_to_24.fits.gz')

hlc=vstack(hlc1,hlc2,hlc3,hlc4,hlc5,hlc6,hlc7,hlc8)


In [ ]:
#--------------NSC----------------------#

nsc1=Table.read('stripe82_315_ra_45_-1_3_dec_0.txt', format='ascii.basic')
nsc2=Table.read('stripe82_315_ra_45_0_dec_1_3.txt', format='ascii.basic')

nsc=vstack(nsc1,nsc2)

print('Done')

In [2]:
idx=Table.read('manxmatch_hlc_nsc_idx.txt', format='ascii.no_header')
dist=Table.read('manxmatch_hlc_nsc_d2d.txt', format='ascii.no_header')

# Crossmatching

In [ ]:
#-----------Read matched indices and distances tables--------------#

idx=Table.read('manxmatch_hlc_nsc_idx.txt', format='ascii.no_header')
dist=Table.read('manxmatch_hlc_nsc_d2d.txt', format='ascii.no_header')

#print(len(idx), len(dist))
#dist['col1']

#----------Make list of duplicates--------------------#

#  Count duplicates
count=collections.Counter(idx['col1'])
#  Turn list into array
count=np.array(count.items())
#  Make mask of multiple matches
multimatch=count[:,1]>1

#-------------Match Multiples in Indices array----------------#

#  Here, I'm going to create a massive list of the indices of matches I want to throw away.

match=[]
b=[]

for indice in count[multimatch][:,0]:
    #Create mask to find multiple indicies
    a=np.where(idx['col1']==indice)[0]
    #Append matches to list
    match.append([a,idx['col1'][a],dist['col1'][a]])
    #Create mask to find the minimum distance within the matches
    b.append(dist['col1'][a]==min(d for d in dist['col1'][a]))


#  Apply the mask to this weird "match" list and make a new list of the wrong 
#  indice matches using the '~'.

wrong=[]

for i in range(len(match)):
   wrong.append(match[i][0][~b[i]])

#  Concatenate list of indicies into "badmatch"
badmatch=np.concatenate(wrong, axis=0)

#---------Check list of the right indice matches--------------#

#right=[]

#for i in range(len(match)):
   #right.append(match[i][0][b[i]])

#---------------------Final list of matches---------------------#
idxnew=np.delete(idx['col1'],badmatch)

#-------------Apply indicies to nsc table-------------#
nsc[idxnew]

#-------------Split nsc table---------------------------------#
nsc1=nsc[0:len(hlc1)-1]
nsc2=nsc[len(hlc1):len(hlc2)-1]
nsc3=nsc[len(hlc2):len(hlc3)-1]
nsc4=nsc[len(hlc3):len(hlc4)-1]
nsc5=nsc[len(hlc4):len(hlc5)-1]
nsc6=nsc[len(hlc5):len(hlc6)-1]
nsc7=nsc[len(hlc6):len(hlc7)-1]
nsc8=nsc[len(hlc7):len(hlc8)-1]

In [84]:
idxnew=np.delete(idx['col1'],badmatch)
len(idxnew)

1527346

(1527346, 1527346)

# Proper Motion

In [156]:
#-----------------dPM----------------------------#
# HLC proper motions
HLCpmRA = 1000*hlc['RA_PM']
HLCpmRAerr = 1000*hlc['RA_PM_ERR']
HLCpmDec = 1000*hlc['DEC_PM']
HLCpmDecerr = 1000*hlc['DEC_PM_ERR']
# NSC proper motions 
NSCpmRA = nsc['pmra']
NSCpmRAerr = nsc['pmraerr']
NSCpmDec = nsc['pmdec']
NSCpmDecerr = nsc['pmdecerr']

dra = 3600*(hlc['RAPM_MEAN'] - nsc['ra']) 
ddec = 3600*(hlc['DEC_MEAN'] - nsc['dec']) 


dPMra = HLCpmRA - NSCpmRA

N, xedges, yedges = binned_statistic_2d(r, , dPMRA, 'count', bins=150)

plt.xlabel('$r$')
plt.ylabel('$dpmRA$')
plt.imshow(np.log10(N.T), origin='lower', extent=[xedges[0], xedges[-1], yedges[0], 
    yedges[-1]], aspect='auto', interpolation='nearest', cmap=multicolor)
    
cb = plt.colorbar(orientation='horizontal')
cb.set_label('$log(count)$')